In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import librosa
import librosa.display
from scipy import fft
from intervaltree import Interval,IntervalTree

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, CategoricalNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report


In [2]:
#test_file = '..musicnet/train_data/1727.wav'
#y, sr = librosa.load(test_file, sr=None)
#y_harmonic, y_percussive = librosa.effects.hpss(y)
#_harmonic.shape
#_harmonic = np.array(y_harmonic)
#_percussive = np.array(y_percussive)

In [3]:
plt.figure(figsize=(10,4))
librosa.display.waveplot(y_harmonic, sr=sr, alpha=0.25)
librosa.display.waveplot(y_percussive, sr=sr, color='r', alpha=0.5)
plt.legend(['Harmonic', 'Percussive'])
plt.title("Amplitude Envelope of Harmonic and Percussive Components for ")
#plt.savefig('plots/soundsAsArrays_'+str(class_names[i])+'.png')
plt.show()

NameError: name 'y_harmonic' is not defined

<Figure size 720x288 with 0 Axes>

In [ ]:
#pd.set_option("display.max_rows", None, "display.max_columns", None)

Run the cell below to run standard training and testing models with 320 training examples and 10 test examples

In [ ]:
# wav file data, metadata labels
train_data = pd.read_csv('../data/df_train_wav_consolidated.csv')
labels = pd.read_csv('../data/musicnet_metadata.csv')
labels.rename({'id': 'filename'}, axis=1, inplace=True)
train = pd.merge(train_data, labels, on =['filename'])
train.drop(['Unnamed: 0', 'filename', 'composer', 'composition', 'movement', 'source', 'transcriber', 'transcriber', 'catalog_name', 'seconds'], axis=1, inplace=True)  # todo: try to include some of these featues
X_train = train.drop('ensemble', axis=1)
y_train = train['ensemble']
code = {}
j = 0
for i in y_train.value_counts().index:
    code[i] = j
    j+=1
y_train.replace(code, inplace=True)

test_data = pd.read_csv('../data/df_test_wav_finc.csv')
test = pd.merge(test_data, labels, on=['filename'])
test.drop(['Unnamed: 0', 'filename', 'composer', 'composition', 'movement', 'source', 'transcriber', 'transcriber', 'catalog_name', 'seconds'], axis=1, inplace=True)
X_test = test.drop('ensemble', axis=1)
y_test = test['ensemble']
y_test.replace(code, inplace=True)

In [ ]:
X_train.shape

Logistic regression:

In [ ]:
lr = LogisticRegression(C = 0.5, solver='liblinear', multi_class='auto')
lr.fit(np.array(X_train), np.array(y_train))
predictions = lr.predict(np.array(X_test))
print('Accuracy score: ', accuracy_score(predictions, np.array(y_test)))
print('F1 Score: ', f1_score(predictions, np.array(y_test), average='weighted'))
print('Classification report: \n', classification_report(y_test, predictions))
print('Confusion Matrix: \n', confusion_matrix(y_test, predictions))

Naive bayes

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, np.array(y_train))
predictions = gnb.predict(X_test)
print('Accuracy score: ', accuracy_score(predictions, np.array(y_test)))
print('F1 Score: ', f1_score(predictions, np.array(y_test), average='weighted'))
print('Classification report: \n', classification_report(y_test, predictions))
print('Confusion Matrix: \n', confusion_matrix(y_test, predictions))

SVM

In [ ]:
svm = SVC()
svm.fit(X_train, y_train)
predictions = dt.predict(X_test)
print('Accuracy score: ', accuracy_score(predictions, np.array(y_test)))
print('F1 Score: ', f1_score(predictions, np.array(y_test), average='weighted'))
print('Classification report: \n', classification_report(y_test, predictions))
print('Confusion Matrix: \n', confusion_matrix(y_test, predictions))

Decision Tree

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
predictions = dt.predict(X_test)
print('Accuracy score: ', accuracy_score(predictions, np.array(y_test)))
print('F1 Score: ', f1_score(predictions, np.array(y_test), average='weighted'))
print('Classification report: \n', classification_report(y_test, predictions))
print('Confusion Matrix: \n', confusion_matrix(y_test, predictions))

Models run with cross validation

Run the cell below to prep data for cross validation X - 330 feature vectors, y - 330 labels

In [ ]:
# wav file data, metadata labels, all in one dataframe for use with cross validation
train_data = pd.read_csv('csv_data/df_train_wav_consolidated.csv')
labels = pd.read_csv('csv_data/musicnet_metadata.csv')
labels.rename({'id': 'filename'}, axis=1, inplace=True)
train = pd.merge(train_data, labels, on =['filename'])
train.drop(['Unnamed: 0', 'filename', 'composer', 'composition', 'movement', 'source', 'transcriber', 'transcriber', 'catalog_name', 'seconds'], axis=1, inplace=True)  # todo: try to include some of these featues
X_train = train.drop('ensemble', axis=1)
y_train = train['ensemble']
code = {}
j = 0
for i in y_train.value_counts().index:
    code[i] = j
    j+=1
y_train.replace(code, inplace=True)

test_data = pd.read_csv('csv_data/df_test_wav_finc.csv')
test = pd.merge(test_data, labels, on=['filename'])
test.drop(['Unnamed: 0', 'filename', 'composer', 'composition', 'movement', 'source', 'transcriber', 'transcriber', 'catalog_name', 'seconds'], axis=1, inplace=True)
X_test = test.drop('ensemble', axis=1)
y_test = test['ensemble']
y_test.replace(code, inplace=True)

X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

Run the cell below to run cross validation

USE THIS CELL BELOW!!!

In [ ]:
# linear regression
lr = LogisticRegression(C = 0.5, solver='liblinear', multi_class='auto')
lr.fit(np.array(X_train), np.array(y_train))
scores = cross_val_score(estimator = lr, X=X, y=y, cv=5, n_jobs=1)
print('Linear regression')
print('CV Scores: ', scores)
print('CV Average and standard deviation: ', np.mean(scores), '+/-', np.std(scores))
# naive bayes
gnb = GaussianNB()
gnb.fit(X_train, np.array(y_train))
scores = cross_val_score(estimator = gnb, X=X, y=y, cv=5, n_jobs=1)
print('Naive Bayes')
print('CV Scores: ', scores)
print('CV Average and standard deviation: ', np.mean(scores), '+/-', np.std(scores))
# SVM
svm = SVC()
svm.fit(X_train, y_train)
scores = cross_val_score(estimator = svm, X=X, y=y, cv=5, n_jobs=1)
print('SVM')
print('CV Scores: ', scores)
print('CV Average and standard deviation: ', np.mean(scores), '+/-', np.std(scores))
# decision tree
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
scores = cross_val_score(estimator = dt, X=X, y=y, cv=5, n_jobs=1)
print('Decision Tree')
print('CV Scores: ', scores)
print('CV Average and standard deviation: ', np.mean(scores), '+/-', np.std(scores))